# ML Final Project JN - Holistic Regresssion

# Credit to  https://github.com/MichaelLLi/ for implementing this originally in Julia 0.6. 

## NOTE: This is a stable final version of the code.

In [ ]:
using ScikitLearn, JuMP, Gurobi, Optim, Distributions, Ipopt, StatsBase,
CSV, LinearAlgebra,Rmath, Arpack, Random, DataFrames, StatsPlots

In [ ]:
@sk_import preprocessing: normalize
@sk_import preprocessing: StandardScaler
@sk_import preprocessing: MinMaxScaler
@sk_import datasets: make_regression

In [ ]:
#Set paths to data
park_path   = "/Users/johnnicholas/desktop/ML/Hw/Hw3/parkinsons_updrs1.csv"
admit_path  = "/Users/johnnicholas/desktop/ML/project/code/admit.csv"
boston_path = "/Users/johnnicholas/desktop/ML/project/code/Boston.csv"
gables_path = "/Users/johnnicholas/desktop/ML/project/code/CoralGablesHouses.csv"

In [ ]:
#PARKINSONS FROM HW3
park = CSV.read(park_path);
X_park = park[:, 1:16];
y_park = park[:, 17];
#ADMIT FROM OLD
admit = CSV.read(admit_path)
X_admit = admit[:,3:9];
y_admit = admit[:,10];
#BOSTON DATASET
boston = CSV.read(boston_path, header=true)[:,2:15]
X_bos = boston[:,1:13];
y_bos = boston[:,14];
#GABLES HOUSING
gables = CSV.read(gables_path, header=true);
mid = hcat(gables[:,1],gables[:,7:10]);
X_gables = hcat(mid, gables[:,3:6]);
y_gables = gables[:,2];

In [ ]:
function MeanImpute(X)
    X_mean_impute = deepcopy(X)
    for d in 1:size(X, 2)
      idx_missing = ismissing.(X[:,d])
      X_mean_impute[idx_missing, d] .= mean(X[.!idx_missing, d])  
    end    
return X_mean_impute
end

In [ ]:
function HighCorrelation(X,HC)
   X = convert(Matrix, X)
corr = cor(X)
    z = ones(1,2)
for i = 1:size(corr)[1]
           for j = i:size(corr)[1]
    if abs(corr[i,j]) >= HC && i!=j 
            q=[i,j]'
            z=vcat(z,q)    
            end
        end
    end  
  return round.(Int,z[2:size(z)[1],1:2])
end

In [ ]:
function betaUB(X,Y,UB,i)
    p=size(X)[2]
    LB=Model(solver=GurobiSolver(OutputFlag=0))
    @variable(LB,β[1:p])
    @constraint(LB,norm(Y-X*β)<=UB)
    @objective(LB,Max,β[i])
    solve(LB)
    β_opti=getvalue(β)
    return β_opti[i]
end

In [ ]:
function betaLB(X,Y,UB,i)
    p=size(X)[2]
    LB=Model(solver=GurobiSolver(OutputFlag=0))
    @variable(LB,β[1:p])
    @constraint(LB,norm(Y-X*β)<=UB)
    @objective(LB,Min,β[i])
    solve(LB)
    β_opti=getvalue(β)
    return β_opti[i]
end

In [ ]:
function findobjUB(X,Y)
p=size(X)[2]
    function naiveobjective(β)
        obj=norm(Y-X*β,2)
        return obj
    end
initial_x=Array{Float64}(undef, p)
initial_x.=0
res=Optim.optimize(naiveobjective, initial_x, NelderMead(),Optim.Options(time_limit = 10.0))
obj_UB=Optim.minimum(res)
β_UB=Optim.minimizer(res)
return obj_UB, β_UB
end

In [ ]:
function findM(X,Y)
    Y= convert(Array, Y)
    X = convert(Matrix, X)
    obj_UB, β_UB=findobjUB(X,Y)
    println("Upper bound of Objective is $obj_UB")
    p=size(X)[2]
    UBArray=Array{Float64}(undef, p)
    LBArray=Array{Float64}(undef, p)
    for i=1:p
        UBArray[i]=betaUB(X,Y,obj_UB,i)
        LBArray[i]=betaLB(X,Y,obj_UB,i)
    end
    return max(maximum(UBArray),minimum(LBArray))
end

In [ ]:
function NLTransform(X)
    X = convert(Matrix, X)
    logX=log.(X)
    sqrtX=sqrt.(X)
    sqX=X.*X
    oldColumn=size(X)[2]
    X=hcat(X,logX,sqrtX,sqX)
    NLArray=Matrix{Int64}(I,oldColumn,4)
    for i=1:oldColumn
            NLArray[i,1]=i
            NLArray[i,2]=i+oldColumn
            NLArray[i,3]=i+oldColumn*2
            NLArray[i,4]=i+oldColumn*3
    end
    X = replace!(X, -Inf=>0)
    X = replace!(X, Inf=>0)
    return NLArray, X
end

In [ ]:
function GlobalMCor(x,delta)    
    X = convert(Matrix, x)
    cols = eigvals(X'*X).<= 0.01
    v = eigvecs(X'*X)[:,cols]
    p = size(v)[1] 
    m = size(v)[2] 
    partd = Model(solver=GurobiSolver(TimeLimit=45, OutputFlag=0))
    # Optimization variables
    @variable(partd, z[j=1:p], Bin)
    @variable(partd, d[i=1:m], Bin)
    @variable(partd, y[h=1:2], Bin)
    @variable(partd, a[j=1:p])
    @variable(partd, t[i=1:m])
    @variable(partd, b)
    # Objective
    @objective(partd, Min, sum(z))
    # Constraints
    @constraint(partd, [j=1:p], 500*z[j] >= a[j])
    @constraint(partd, [j=1:p], -500*z[j] <= a[j])
    @constraint(partd, sum(t[i].*v[:,i] for i in 1:m, dims=2) .== a)
    @constraint(partd, b == sum(t))
    @constraint(partd, b <= -delta + (500)*(1-y[1]))
    @constraint(partd, b >= delta + (-500)*(1-y[2]))
    @constraint(partd, y[1] + y[2] == 1)
    solve(partd)
    status = solve(partd)
S=[]    
while status != "INFEASIBLE"
supp_a = findall(x -> x >= 0.001, getvalue(z))
        
  if iszero(supp_a) == false
    push!(S, supp_a)
    @constraint(partd, sum(z[i] for i in supp_a) <= length(supp_a)-1)
    solve(partd)
    status = solve(partd) 
  else 
    break
  end      
end
    if size(S,1)<=1
        S=zeros(1,(size(X,2)))
    else
    end
S = filter(x -> size(x,1) >=2 ,S)   
 return S   
end

In [ ]:
function Normalize(X, Y)
scaler =  MinMaxScaler()
        X = ScikitLearn.fit_transform!(scaler,X)
        Y = ScikitLearn.fit_transform!(scaler,convert(Matrix,hcat(Y,ones(size(Y,1),1))))[:,1]
    return X,Y
end

In [ ]:
function OptSplit(vector,trainp)
model = Model(solver=GurobiSolver(TimeLimit=120, OutputFlag=0))
    full_size = size(vector,1)
    train_size=round(Int,size(vector,1)*trainp)
    one1 = vector
    rho=1
    # Optimization variables
    h = size(one1,1)
    @variable(model, x[i=1:h,p=1:2], Bin)
    @variable(model, d>=0)
    
    # Objective
    @objective(model, Min, d)
    
    
    # Constraints
    #u1 - u2 + o1 - o2
    @constraint(model, d >= (1/10)*sum(one1[i]*x[i,1] for i in 1:h) - (1/10)*sum(one1[i]*x[i,2] for i in 1:h)
                       +rho*(1/10)*sum((one1[i]^2)*x[i,1] for i in 1:h) - rho*(1/10)*sum((one1[i]^2)*x[i,2] for i in 1:h))

    #u1 - u2 - o1 + o2
    @constraint(model, d >= (1/10)*sum(one1[i]*x[i,1] for i in 1:h) - (1/10)*sum(one1[i]*x[i,2] for i in 1:h)
                        -rho*(1/10)*sum((one1[i]^2)*x[i,1] for i in 1:h) + rho*(1/10)*sum((one1[i]^2)*x[i,2] for i in 1:h))




    #-u1 + u2 + o1 - o2
     @constraint(model, d >= -(1/10)*sum(one1[i]*x[i,1] for i in 1:h)+(1/10)*sum(one1[i]*x[i,2] for i in 1:h)
                          +rho*(1/10)*sum((one1[i]^2)*x[i,1] for i in 1:h) - rho*(1/10)*sum((one1[i]^2)*x[i,2] for i in 1:h)) 


    #-u1 + u2 - o1 + o2
    @constraint(model, d >= -(1/10)*sum(one1[i]*x[i,1] for i in 1:h)+(1/10)*sum(one1[i]*x[i,2] for i in 1:h)
                        -rho*(1/10)*sum((one1[i]^2)*x[i,1] for i in 1:h) + rho*(1/10)*sum((one1[i]^2)*x[i,2] for i in 1:h))
    
@constraint(model, sum(x[i,1] for i in 1:h) == train_size)
@constraint(model, sum(x[i,2] for i in 1:h) == abs(train_size-full_size))
@constraint(model, [i=1:h], sum(x[i,p] for p in 1:2) == 1)   
     solve(model)

  return getvalue(d), getvalue(x)
end

In [ ]:
function OptSplitWrapper(X,vector, trainp)
    full_size = size(vector,1)
    y = OptSplit(vector,trainp)
    first_grp  = filter( y -> abs(y)>=0.000000001, (round.(Int,y[2][1:full_size]).*vector));
    second_grp = filter( y -> abs(y)>=0.000000001, (round.(Int,y[2][full_size+1:2*full_size]).*vector));

    #Recover X_train from OptSplit y_train
    admit_c = convert(DataFrame,hcat(convert(Matrix,X),convert(Vector,vector)))
    X_c = convert(DataFrame,copy(X))
    X_group1 = ones(1,size(X,2))
    for i in 1:size(first_grp,1)
        j = findall(x-> x==first_grp[i],admit_c[:,size(admit_c,2)])[1]
        X_group1 = vcat(X_group1 , convert(Vector,X_c[j,:])')
        deleterows!(admit_c, j)
        deleterows!(X_c, j)
    end
    X_train = X_group1[2:size(X_group1,1),:]
    #END
    
    #Recover X_test from OptSplit y_train
    X_group2 = ones(1,size(X,2))
    for i in 1:size(second_grp,1)
        j = findall(x-> x==second_grp[i],admit_c[:,size(admit_c,2)])[1]
        X_group2 = vcat(X_group2 , convert(Vector,X_c[j,:])')
        deleterows!(admit_c, j)
        deleterows!(X_c, j)
    end
    X_test = X_group2[2:size(X_group2,1),:]
    #END
    
    return first_grp, second_grp, X_train, X_test
end

# Now comes regression methods

In [ ]:
function OptimRegression(X,Y,X_valid,Y_valid,X_test,Y_test,invX,vari,HCArray,HC,M,NLArray,sign,MCArray,NLtrue=false, Signtrue=false,returnbeta=false, MCtrue=false)
    function RegressionWrapper(x)
        obj, β=SystematicRegression(X,Y,invX,vari,x[1],x[2],HCArray,HC,M,NLArray,sign,MCArray,NLtrue,Signtrue,true, MCtrue)
        β[findall(abs.(β).<1e-5)].=0
        obj_valid=norm(Y_valid-X_valid*β,2)
        return obj_valid
    end
    initial_x=[0.1,5]
    println("Optimizer Starting")
    res=Optim.optimize(RegressionWrapper, initial_x, NelderMead(),Optim.Options(f_calls_limit=20))
    opti_HP=Optim.minimizer(res)
    obj_valid=Optim.minimum(res)
    obj_opti,β_opti=SystematicRegression(X,Y,invX,vari,opti_HP[1],opti_HP[2],HCArray,HC,M,NLArray,sign,MCArray,NLtrue,Signtrue,true,MCtrue)
    β_opti[findall(abs.(β_opti).<1e-5)].=0
    obj_test=norm(Y_test-X_test*β_opti,2)
    return β_opti,obj_valid,obj_test,opti_HP[1],opti_HP[2]
end

In [ ]:
function SystematicRegression(X,Y,invX,vari,Γ,k,HCArray,HC,M,NLArray,sign,MCArray,NLtrue=false, Signtrue=false,returnbeta=false,MCtrue=false)
# If we want to also test significance, write Signtrue=true
# sign is the t-value threshold you want to test
# if you want beta values to be returned, take returnbeta as true
if Γ<=0
    Γ=0
end
if Γ>=1
    Γ=1
end
if k>=size(X)[2]-size(NLArray)[1]*3
    k=size(X)[2]-size(NLArray)[1]*3
end
if k<=1
    k=1
end
M2=100000
model=Model(solver=GurobiSolver(OutputFlag=0,NumericFocus=2))
if size(X)[1] !=size(Y)[1]
    throw(ArgumentError("check sizes of array X and Y"))
end
n=size(X)[1]
p=size(X)[2]
@variable(model,z[1:p],Bin)
@variable(model,β[1:p])
@variable(model,l[1:p])
@constraint(model,[i=1:p], l[i]>=β[i])
@constraint(model,[i=1:p], l[i]>=-β[i])
@constraint(model,[i=1:p], M*z[i]>=β[i])
@constraint(model,[i=1:p], M*z[i]>=-β[i])
@constraint(model,sum(z[i] for i=1:p)<=k) #Sparsity
if size(HCArray)[1]>0
    for i=1:size(HCArray)[1]
        @constraint(model,sum(z[HCArray[i,j]] for j=1:size(HCArray)[2])<=1) #pairwise collinearity
    end
end
if NLtrue
    for i=1:size(NLArray)[1]
        @constraint(model,sum(z[NLArray[i,j]] for j=1:size(NLArray)[2])<=1) # Nonlinear transformation
    end
end
if Signtrue
    if NLtrue
        @variable(model,b[1:p],Bin)
        for i=1:p
            k=mod(i,p/4)
            if k==0
                k=p/4
            end
            k=convert(Int64,k)
            c=(1)./sqrt(vari*invX[k,k])
            @constraint(model,β[i]*c[1]+M2*c[1]*b[i]>=(quantile(TDist(n-p),1-sign))*z[i]) #significance
            @constraint(model,β[i]*c[1]+M2*c[1]*b[i]<=M2*c[1]-(quantile(TDist(n-p),1-sign))*z[i])  #significance
        end
    else
        @variable(model,b[1:p],Bin)
        for i=1:p
            c=(1)./sqrt(vari*invX[i,i])
            @constraint(model,β[i]*c[1]+M2*c[1]*b[i]>=(quantile.(TDist(n-p),1-sign*0.5))*z[i]) #significance
            @constraint(model,β[i]*c[1]+M2*c[1]*b[i]<=M2*c[1]-(quantile.(TDist(n-p),1-sign*0.5))*z[i]) #significance
        end
    end
end
if MCtrue
      if size(MCArray,1)>0                     
       for i in 1:size(MCArray,1)
        @constraint(model,sum(z[MCArray[i][j]] for j in 1:size(MCArray[i],1)) <= (size(MCArray[i],1)))
       end
      end
end
@objective(model, Min, sum((Y[i]-(X*β)[i])*(Y[i]-(X*β)[i]) for i=1:n)/2+sum(l[i] for i=1:p)Γ)
solve(model)
β_opti=getvalue(β)
objective=getobjectivevalue(model)
if returnbeta
    return objective, β_opti
else
    return objective
end
end

In [ ]:
function SRegression2(X,Y,HC,NLtrue,sign,Signtrue,Normtrue,MCtrue,Imputetrue) 
      #Normalization/Imputation
    if Normtrue 
       nrm = Normalize(X,Y)
        X = nrm[1]
        Y = nrm[2]
    end
     println("Normalization complete")
     
    if Imputetrue
    X = MeanImpute(X)
    end
   
    
    #There is no random/optimal splitting done in here, just splitting up what's passed in.
    train=round(Int,size(X)[1]*0.6)
    test=round(Int,size(X)[1]*0.8)
     
  
    X_train_1=X[1:train,:]
    Y_train_1=Y[1:train]
    
    M=findM(X_train_1,Y_train_1)
    println("M found: $M")
    invX=inv(X_train_1'X_train_1)
    P=X_train_1*(invX)X_train_1'
    n=size(X_train_1)[1]
    p=size(X_train_1)[2]
    RSS=Y_train_1'*(I(n)-P)Y_train_1
    vari=RSS/(n-p)
    println("Significance Calculated")
    if NLtrue && !Normtrue
        println("ERROR if NLTRUE, THEN ALSO MAKE NORM TRUE")
    elseif NLtrue
        NLArray, X=NLTransform(X)
    else
        NLArray = zeros(0,0)
    end
    X_train=X[1:train,:]
    Y_train=Y[1:train]
    X_valid=X[train+1:test,:]
    Y_valid=Y[train+1:test]
    X_test=X[test:end,:]
    Y_test=Y[test:end]
    println("Split completed")
    HCArray=HighCorrelation(X_train,HC)
    println("HCArray found")
    MCArray = GlobalMCor(X_train,0.015)
    MCArraySize=size(MCArray,1)
    println("MCArray found, Size: $MCArraySize")
    println(MCArray)
    β, obj_valid,obj_test, Γ, k=OptimRegression(X_train,Y_train,X_valid,Y_valid,X_test,Y_test,invX,vari,HCArray,HC,M,NLArray,sign,MCArray,NLtrue,Signtrue,false,MCtrue)
    return β, obj_valid,obj_test, Γ, k
end

# Now wrote Holistic Regression Wrapper that just takes data & T/F

In [ ]:
function HolisticRegression(X,Y,HC=0.99,NLtrue=false,sign=0.95,Signtrue=false,Normtrue=false,
                             MCtrue=false,OptSplit=false,Imputetrue=false,trainp=0.8)
    
    #Conversion to array if not an array
    if typeof(X) != Array
      X=convert(Matrix{Float64},X)
    end
    if typeof(X) != Array
      Y=convert(Vector{Float64},Y)
    end
    #End
    
    #Split into train and test
    if OptSplit == true
        groups = OptSplitWrapper(X,Y, trainp)
        Y_train = groups[1] 
        Y_test = groups[2]
        X_train = groups[3]
        X_test = groups[4]
    println("Optimal data split calculated")
          
    else
      Random.seed!(4567)
      index = Distributions.sample(1:size(X,1),size(X,1),replace=false);
      train=round(Int,size(X,1)*trainp)
      X_train = X[1:train,:]
      X_test =  X[train+1:size(X,1),:]
      Y_train = Y[1:train,:]
      Y_test =  Y[train+1:size(X,1),:]
    end
    
    if NLtrue == true
    results = SRegression2(X_train,Y_train,HC,NLtrue,sign,Signtrue,Normtrue,MCtrue,Imputetrue)
    #IN SAMPLE RESULTS
        scaler =  MinMaxScaler()
        x = NLTransform(ScikitLearn.fit_transform!(scaler,X_train))
    #x = NLTransform(normalize(X_train))
    y_hat = x[2]*results[1];
     y = ScikitLearn.fit_transform!(scaler,convert(Matrix,hcat(Y_train,ones(size(Y_train,1),1))))[:,1]
    #y = normalize(convert(Matrix,hcat(Y_train,ones(size(Y_train,1),1))))[:,1]
     #y = convert(Array,Y_train)
    SSE1 = sum((y_hat.-y).^2)
    SST = sum((mean(y).-y).^2)
    In_Sample_Rsq = (1-(SSE1/SST))
    In_Sample_RMSE = sqrt(mean(((y_hat).-y).^2))
    #OUT OF SAMPLE RESULTS
    x = NLTransform(ScikitLearn.fit_transform!(scaler,X_test))
    #x = NLTransform(normalize(X_test))
    #x = NLTransform(X_test)
    y_hat = x[2]*results[1];
      #y = convert(Array,Y_test)
    #y = normalize(convert(Matrix,hcat(Y_test,ones(size(Y_test,1),1))))[:,1]
    y = ScikitLearn.fit_transform!(scaler,convert(Matrix,hcat(Y_test,ones(size(Y_test,1),1))))[:,1]
    SSE1 = sum((y_hat.-y).^2)
    SST = sum((mean(y).-y).^2)
    Out_Sample_Rsq = (1-(SSE1/SST))
    Out_Sample_RMSE = sqrt(mean(((y_hat).-y).^2))
    results = results[1]
    
    else
    results = SRegression2(X_train,Y_train,HC,NLtrue,sign,Signtrue,Normtrue,MCtrue,Imputetrue)
    #IN SAMPLE RESULTS
    x = convert(Matrix,X_train)
    y_hat = x*results[1];
    y = convert(Array, Y_train)
    SSE1 = sum((y_hat.-y).^2)
    SST = sum((mean(y).-y).^2)
    In_Sample_Rsq = (1-(SSE1/SST))
    In_Sample_RMSE = sqrt(mean(((y_hat).-y).^2))
    #OUT OF SAMPLE RESULTS
    xt = convert(Matrix,X_test)
    y_hat = xt*results[1];
    y = convert(Array, Y_test)
    SSE2 = sum((y_hat.-y).^2)
    SST = sum((mean(y).-y).^2)
    Out_Sample_Rsq = (1-(SSE2/SST))
    Out_Sample_RMSE = sqrt(mean(((y_hat).-y).^2))   
    end

println("In sample R^2: $In_Sample_Rsq")
println("Out of sample R^2: $Out_Sample_Rsq")
println("In sample RMSE: $In_Sample_RMSE")
println("Out of sample RMSE: $Out_Sample_RMSE ")
return results
end

# Tests

In [ ]:
#(X,Y,HC=0.99,NLtrue=false,sign=0.95,Signtrue=false,Normtrue=false,
                             #MCtrue=false,OptSplit=false,Imputetrue=false,trainp=0.8)
HolisticRegression(X_admit, y_admit,0.99, true, 0.95, true, true, true, false, false, 0.8)

In [ ]:
#(X,Y,HC=0.99,NLtrue=false,sign=0.95,Signtrue=false,Normtrue=false,
                             #MCtrue=false,OptSplit=false,Imputetrue=false,trainp=0.8)
HolisticRegression(X_gables, y_gables, 0.99, true, 0.95, true, true, true, true,false,0.8)